In [ ]:
import math, random
from typing import Callable, Literal, NamedTuple, Sequence

import ipywidgets as W
from IPython.display import display
import time

Mark = Literal['X','O']
Cell = Literal['X','O',None]

class GameState(NamedTuple):
    board: tuple[Cell, ...]
    to_move: Mark
    winner: Cell
    terminal: bool

# ---------- Game logic ----------
WIN_LINES = [(0,1,2),(3,4,5),(6,7,8),
             (0,3,6),(1,4,7),(2,5,8),
             (0,4,8),(2,4,6)]

def check_winner(board: Sequence[Cell]) -> Cell:
    for a,b,c in WIN_LINES:
        if board[a] and board[a] == board[b] == board[c]:
            return board[a]
    return None

def available_moves(board: Sequence[Cell]) -> list[int]:
    return [i for i,v in enumerate(board) if v is None]

def is_terminal(board: Sequence[Cell]) -> bool:
    return check_winner(board) is not None or not available_moves(board)

def next_player(mark: Mark) -> Mark:
    return 'O' if mark == 'X' else 'X'

def make_move(board: Sequence[Cell], idx: int, mark: Mark) -> tuple[Cell, ...]:
    if board[idx] is not None:
        raise ValueError("Illegal move")
    b = list(board)
    b[idx] = mark
    return tuple(b)

def get_state(board: Sequence[Cell], to_move: Mark) -> GameState:
    w = check_winner(board)
    return GameState(tuple(board), to_move, w, is_terminal(board))

HUMAN = "HUMAN"

def random_ai(board: tuple[Cell, ...], mark: Mark) -> int:
    return random.choice(available_moves(board))

def minimax_ai(board: tuple[Cell, ...], mark: Mark) -> int:
    me: Mark = mark
    opp: Mark = next_player(me)

    def score(b: tuple[Cell, ...]) -> int:
        w = check_winner(b)
        if w == me:  return 1
        if w == opp: return -1
        return 0

    def ab(b: tuple[Cell, ...], to_move: Mark, alpha: int, beta: int) -> int:
        if is_terminal(b): 
            return score(b)
        if to_move == me:
            val = -math.inf
            for m in available_moves(b):
                val = max(val, ab(make_move(b, m, to_move), next_player(to_move), alpha, beta))
                alpha = max(alpha, val)
                if beta <= alpha: 
                    break
            return val
        else:
            val = math.inf
            for m in available_moves(b):
                val = min(val, ab(make_move(b, m, to_move), next_player(to_move), alpha, beta))
                beta = min(beta, val)
                if beta <= alpha: 
                    break
            return val

    best_val = -math.inf
    best_moves: list[int] = []
    for m in available_moves(board):
        v = ab(make_move(board, m, me), next_player(me), -math.inf, math.inf)
        if v > best_val:
            best_val = v
            best_moves = [m]
        elif v == best_val:
            best_moves.append(m)

    # choose randomly among equally good moves
    assert best_moves, "At least one legal move must exist."
    return random.choice(best_moves)

# Registry of available players
PLAYER_FACTORIES: dict[str, Callable[[tuple[Cell,...], Mark], int] | Literal["HUMAN"]] = {
    "Human": HUMAN,
    "AI - Random": random_ai,
    "AI - Minimax (perfect)": minimax_ai,
}

# ---------- UI ----------

class TicTacToeWidget:
    def __init__(self):
        self.board: tuple = tuple([None]*9)
        self.to_move: str = 'X'
        self.lock_human_input = False

        # Board UI
        self.btns = W.GridspecLayout(
            3, 3,
            grid_gap="0px",
            layout=W.Layout(
                width="180px",  # 3 * 50px
                height="180px", # 3 * 50px
                grid_template_rows="50px 50px 50px",
                grid_template_columns="50px 50px 50px",
                margin="0px",
                padding="0px"
            )
        )       
        self.status = W.HTML(value="")
        self.turn = W.HTML(value="")

        # Game controls
        self.reset_btn = W.Button(description="New game", button_style="")
        self.starting_mark = W.ToggleButtons(options=[('X starts','X'),('O starts','O')], value='X')
        self.px = W.Dropdown(options=list(PLAYER_FACTORIES.keys()), value="Human", description="X:")
        self.po = W.Dropdown(options=list(PLAYER_FACTORIES.keys()), value="AI - Minimax (perfect)", description="O:")
        self.auto_play = W.ToggleButton(value=False, description='Auto-play (AI vs AI)', icon='')

        # Simulation controls
        self.sim_runs = W.IntText(value=100, description="Number of games:", step=100, layout=W.Layout(width="200px"))
        self.sim_btn = W.Button(description="Simulate N games (AI vs AI)", button_style="primary")
        self.sim_out = W.Output()

        self.layout = W.VBox([
            W.HBox([self.px, self.po, self.starting_mark, self.reset_btn, self.auto_play]),
            self.turn,
            self.btns,
            self.status,
            W.HTML("<hr>"),
            W.HBox([self.sim_runs, self.sim_btn]),
            self.sim_out
        ])

        self._wire_buttons()
        self.reset_btn.on_click(self._on_reset)
        self.auto_play.observe(self._on_autoplay_toggle, names='value')
        self.sim_btn.on_click(self._on_simulate_click)
        self._render()

    def _wire_buttons(self):
        for r in range(3):
            for c in range(3):
                i = r*3+c
                b = W.Button(description=" ", layout=W.Layout(width="50px", height="50px", margin="5px", padding="5px"))
                b.style.button_color = "#f5f5f5"
                b.add_class('ttt-cell')
                b.on_click(lambda _b, i=i: self._on_cell_click(i))
                self.btns[r, c] = b

    def display(self):
        display(self.layout)
        self._maybe_ai_move()

    # --- State & rendering ---
    def _state(self):
        return get_state(self.board, self.to_move)

    def _render(self):
        st = self._state()
        for i in range(9):
            val = self.board[i]
            btn = self.btns[i//3, i%3]
            # keep a space for empty cells so they visually clear
            btn.description = " " if val is None else val
            btn.disabled = (
                self.lock_human_input
                or st.terminal
                or (val is not None)
            )
        if st.terminal:
            if st.winner:
                self.turn.value = f"<b>Game over.</b>"
                self.status.value = f"<b>{st.winner} wins.</b>"
            else:
                self.turn.value = f"<b>Game over.</b>"
                self.status.value = "<b>Draw.</b>"
        else:
            self.turn.value = f"Turn: <b>{self.to_move}</b>"
            self.status.value = "&nbsp;"

    def _apply_move(self, idx: int):
        self.board = make_move(self.board, idx, self.to_move)
        self.to_move = next_player(self.to_move)
        self._render()

    # --- Human input ---
    def _on_cell_click(self, idx: int):
        st = self._state()
        if st.terminal or self.lock_human_input:
            return
        current_player_kind = self._player_kind(self.to_move)
        if current_player_kind is HUMAN:
            if self.board[idx] is None:
                self._apply_move(idx)
                self._maybe_ai_move()

    # --- Reset / Auto-play ---
    def _on_reset(self, _):
        self.board = tuple([None]*9)
        self.to_move = self.starting_mark.value  # 'X'/'O'
        self.lock_human_input = False
        # explicitly clear button labels
        for i in range(9):
            self.btns[i//3, i%3].description = " "
        self._render()
        self._maybe_ai_move()

    def _on_autoplay_toggle(self, change):
        if change['new'] is True:
            self._maybe_ai_move()

    # --- Player type & AI moves ---
    def _player_kind(self, mark: str):
        return PLAYER_FACTORIES[self.px.value if mark == 'X' else self.po.value]

    def _choose_ai_move(self, mark: str) -> int:
        chooser = self._player_kind(mark)
        assert chooser is not HUMAN
        return chooser(self.board, mark)

    def _maybe_ai_move(self):
        while not self._state().terminal:
            kind = self._player_kind(self.to_move)
            if kind is HUMAN:
                self.lock_human_input = False
                self._render()
                return
            self.lock_human_input = True
            self._render()
            mv = self._choose_ai_move(self.to_move)
            self._apply_move(mv)
            if not self.auto_play.value and self._player_kind(self.to_move) is HUMAN:
                break
        self.lock_human_input = False
        self._render()

    # --- AI vs AI Simulation ---
    def _resolve_ai_callable(self, label: str):
        chooser = PLAYER_FACTORIES[label]
        if chooser is HUMAN:
            # If a human is selected, use perfect AI instead for simulation
            return PLAYER_FACTORIES["AI - Minimax (perfect)"]
        return chooser

    def _play_one_ai_game(self, ai_x, ai_o, starting: str):
        board = tuple([None]*9)
        to_move = starting
        total_time = {'X': 0.0, 'O': 0.0}
        moves_count = {'X': 0, 'O': 0}

        while True:
            w = check_winner(board)
            if w is not None:
                return w, total_time, moves_count
            if not available_moves(board):
                return None, total_time, moves_count  # draw

            ai = ai_x if to_move == 'X' else ai_o
            t0 = time.perf_counter()
            mv = ai(board, to_move)
            dt = time.perf_counter() - t0
            total_time[to_move] += dt
            moves_count[to_move] += 1

            board = make_move(board, mv, to_move)
            to_move = next_player(to_move)

    def _on_simulate_click(self, _):
        N = max(1, int(self.sim_runs.value))

        ai_x = self._resolve_ai_callable(self.px.value)
        ai_o = self._resolve_ai_callable(self.po.value)

        wins_X = wins_O = draws = 0
        agg_time = {'X': 0.0, 'O': 0.0}
        agg_moves = {'X': 0, 'O': 0}

        t_start = time.perf_counter()
        start_mark = self.starting_mark.value

        for g in range(N):
            starting = 'X' if ((start_mark == 'X' and g % 2 == 0) or (start_mark == 'O' and g % 2 == 1)) else 'O'
            w, tms, mvs = self._play_one_ai_game(ai_x, ai_o, starting)
            if w == 'X':
                wins_X += 1
            elif w == 'O':
                wins_O += 1
            else:
                draws += 1
            agg_time['X'] += tms['X']; agg_time['O'] += tms['O']
            agg_moves['X'] += mvs['X']; agg_moves['O'] += mvs['O']

            t_total = time.perf_counter() - t_start
            avg_t_per_move_X = (agg_time['X'] / agg_moves['X']) if agg_moves['X'] else 0.0
            avg_t_per_move_O = (agg_time['O'] / agg_moves['O']) if agg_moves['O'] else 0.0
    
            with self.sim_out:
                self.sim_out.clear_output()
                display(W.HTML(
                    f"""
                    <b>Games simulated:</b> {N}<br>
                    <b>Starting alternation based on setting:</b> {self.starting_mark.value} / alternating<br>
                    <hr>
                    <table>
                      <tr><td><b>X wins</b></td><td>{wins_X} ({wins_X/N:.1%})</td></tr>
                      <tr><td><b>O wins</b></td><td>{wins_O} ({wins_O/N:.1%})</td></tr>
                      <tr><td><b>Draws</b></td><td>{draws} ({draws/N:.1%})</td></tr>
                    </table>
                    <hr>
                    <table>
                      <tr><td></td><td><b>Total time [s]</b></td><td><b>Avg per move [s]</b></td><td><b>Moves</b></td></tr>
                      <tr><td><b>X</b></td><td>{agg_time['X']:.6f}</td><td>{avg_t_per_move_X:.6f}</td><td>{agg_moves['X']}</td></tr>
                      <tr><td><b>O</b></td><td>{agg_time['O']:.6f}</td><td>{avg_t_per_move_O:.6f}</td><td>{agg_moves['O']}</td></tr>
                    </table>
                    <hr>
                    <b>Total simulation time:</b> {t_total:.6f} s
                    """
                ))

# ---------- How to add your own AI players ----------
# Just create function: def my_ai(board: tuple[Cell,...], mark: Mark) -> int
# and register it to PLAYER_FACTORIES, e.g.:
#
# def first_free_ai(board, mark):
#     return [i for i,v in enumerate(board) if v is None][0]
#
# PLAYER_FACTORIES["AI - First Free"] = first_free_ai


In [ ]:
t = TicTacToeWidget()
t.display()